In [2]:
import json
import os
from datetime import datetime
from appdirs import user_data_dir


timestamp = datetime.now().strftime("%Y%m%d-%H-%M-%S")
duration_h = 2
num_motes = 25
algo = 'ori'
pps = 1

use_large = False

app = f"03oos_openwsn_prog-{algo}-{pps}"
site = "lille"
energy_profile = "energym3"

small_nodes = "250-254+238-242" # 5*2 = 10
small_nodes_root = "252"
small_near_root = "240"
large_nodes = "250-255+238-243+225-230+205-210+191-196" # 6*5 = 30 # 169-174+152-157
large_nodes_root = "207"
large_near_root = "208"

if use_large:
    nodes = large_nodes
    root = large_nodes_root
    near_root = large_near_root
else:
    nodes = small_nodes
    root = small_nodes_root
    near_root = small_near_root

In [26]:
file_name = 'exp_log'
def get_log_path(file_name="x", format="txt", folder_only=False):
    if folder_only:
        return user_data_dir('openvisualizer')
    else:
        return os.path.join(user_data_dir('openvisualizer'), file_name+"."+format)

def run_command(cmd):
    val = !{cmd}
    val = [str(i).strip() for i in val]
    result = json.loads("".join(val))
    return result

### Submit experimet

In [ ]:
# Submit experiment
cmd = f"iotlab-experiment submit -n test_{timestamp} -d {int(duration_h*60)} -l {site},m3,{app},{energy_profile}"

result = run_command(cmd)
exp_id = result["id"]

cmd = f"iotlab-experiment wait -i {exp_id}"
run_command(cmd)


In [22]:
node_list = []
for group in nodes.split("+"):
    start, end = group.split("-")
    start = int(start)
    end = int(end)
    gr_ = list(range(start,end+1))
    node_list.extend(gr_)

node_id_list = [f"m3-{n}" for n in node_list]
node_site_list = [f"{n}.{site}.iot-lab.info" for n in node_id_list]

In [24]:
cmd = f"openv-server --iotlab-motes {' '.join(node_site_list)}"

root_site = f"m3-{root}.{site}.iot-lab.info"
cmd = f"openv-client root {root_site}"

cmd = "openv-client view web"

openv-server --iotlab-motes m3-250.lille.iot-lab.info m3-251.lille.iot-lab.info m3-252.lille.iot-lab.info m3-253.lille.iot-lab.info m3-254.lille.iot-lab.info m3-238.lille.iot-lab.info m3-239.lille.iot-lab.info m3-240.lille.iot-lab.info m3-241.lille.iot-lab.info m3-242.lille.iot-lab.info
openv-client root m3-252.lille.iot-lab.info


In [ ]:
# download oml energy
oml_path = get_log_path(folder_only=True)
cmd = f"pscp -r -i ~/.ssh/id_rsa.ppk dfawwaz@saclay.iot-lab.info:.iot-lab/{exp_id}/consumption/ {oml_path}/consumption/"
cmd = f"plot_oml_consum -p -i {oml_path}/consumption/m3-{near_root}.oml"


### Add/Remove mote

In [ ]:
# To remove: Stop and clear flash

# To add: Stop and clear flash, then flash with oos and start

last_node = node_list[-1]
cmd = f"iotlab-node --stop -l {site},m3,{last_node}"
cmd = f"iotlab-node --start -l {site},m3,{last_node}"